In [187]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV,cross_val_score, KFold


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


df = pd.read_csv("DataCSVS/cleanedPlayerData41-42.csv", encoding='latin1')


df.head()



df.columns

Index(['Weight', 'IS', 'OS', 'Rng', 'Fin', 'Reb', 'IDef', 'PDef', 'IQ', 'Pass',
       'Hnd', 'Drv', 'Str', 'Spd', 'Sta', 'F-M', 'F-A', 'F%', 'IS-M', 'IS-A',
       'IS%', 'MR-M', 'MR-A', 'MR%', '3P-M', '3P-A', '3P%', 'DR-M', 'DR-A',
       'DR%', 'FTM', 'FTA', 'FT%', 'RebP', 'Ast', 'Stl', 'Blk', '2OFM', '2OFA',
       '2OF%', '3OFM', '3OFA', '3OF%', 'TO', 'PF', 'DQ', 'FD',
       'Wingspan_inches', 'Height_inches', 'Vertical_float'],
      dtype='object')

In [188]:

df = df[df['2OFA'] >= 0.7 * (df['2OFA'] + df['3OFA'])] #For Bigs
#df = df[df['3OFA'] >= 0.4 * (df['2OFA'] + df['3OFA'])] #For Guards

df = df[df['2OFA'] >= 100] #Filters for players who play a decent amount per game
columns_to_drop = ['F-M', 'F-A', 'F%','IS-M', 'IS-A', 'IS%',
        'MR-M', 'MR-A', 'MR%', '3P-M', '3P-A', '3P%', 'DR-M', 'DR-A',
       'DR%', 'FTM', 'FTA', 'FT%', 'RebP', 'Ast', 'Blk', '2OFM', '2OFA', '2OF%',
        '3OFM', '3OFA', '3OF%', 'TO', 'PF', 'DQ', 'FD']



df = df.drop(columns_to_drop,axis=1)


print(df.shape)

data = df
X = data.drop('Stl', axis=1)
y = data['Stl']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


(2418, 19)


In [189]:
from sklearn.feature_selection import RFE

# RFE with Random Forest
rfe = RFE(estimator=Ridge(), n_features_to_select=5)
rfe.fit(X_train, y_train)

# Selected features
selected_features = X_train.columns[rfe.support_]
print("Selected features:", selected_features)


Selected features: Index(['IDef', 'IQ', 'Str', 'Spd', 'Sta'], dtype='object')


In [190]:
# Transform the training and testing sets
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

#Ridge Regression is best model

model = Ridge()
pipeline = make_pipeline(StandardScaler(), model)

# Hyperparameter Tuning with Grid Search
param_grid = {'ridge__alpha': [0.1, 1.0, 10.0, 100.0]}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_selected, y_train)
best_model = grid_search.best_estimator_

best_model.fit(X_train_selected, y_train)
y_pred = best_model.predict(X_test_selected)

'''
pipeline.fit(X_train_selected, y_train)
y_pred = pipeline.predict(X_test_selected)
'''
# Evaluate the model
r2 = r2_score(y_test, y_pred)
print(f'R2 Score: {r2}')





R2 Score: 0.33929105739662624


In [191]:

import pandas as pd

# Create a DataFrame with actual and predicted values
results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

# Optionally, you can reset the index to make it easier to read
results = results.reset_index(drop=True)

# Display the DataFrame
#print(results.head())  # Shows the first few rows of the DataFrame

# Create a DataFrame with actual features from X_test
results = pd.DataFrame(X_test)

# Add actual and predicted values to this DataFrame
results['Actual'] = y_test
results['Predicted'] = y_pred

# Reset the index to align the actual and predicted values with their corresponding features
results = results.reset_index(drop=True)

# Display the DataFrame
print(results.head())  # Shows the first few rows of the DataFrame






   Weight  IS  OS  Rng  Fin  Reb  IDef  PDef  IQ  Pass  Hnd  Drv  Str  Spd  \
0     235   0   3    8    1    8     3     3   6     3    4    2    3    1   
1     190   8  14   14   13   16    13    10   8     6    5    8   17    6   
2     230   9   7    7    8   11     8     4   4     8    5    5    6    5   
3     225  16  14   12    7   12     7    12  12     7   11    8   15   18   
4     255   7  11    7    5   11     4     8  15     3    7    9   13   10   

   Sta  Wingspan_inches  Height_inches  Vertical_float  Actual  Predicted  
0    2             85.0           79.5            30.5     0.1  -0.080233  
1   13             79.0           76.5            30.5     0.4   0.869869  
2    5             89.0           82.5            29.0     0.1   0.270397  
3    9             82.0           78.0            30.5     0.9   0.941759  
4    6             91.0           82.0            29.0     0.8   0.591969  


In [192]:
# Code that Finds a player based on their PLayer Code and Returns their skills (uses bs4)

from pygetPlayerSkills import get_player_info

player  = get_player_info("http://onlinecollegebasketball.org/player/178675")

new_data_df = pd.DataFrame([player])

# Select only the features that were used for training
new_data_selected = new_data_df[selected_features]

'''
# Predict using the model pipeline
new_data_pred = pipeline.predict(new_data_selected)
'''
new_data_pred = best_model.predict(new_data_selected)

# Print the predictions
print(format(new_data_pred[0],".2f"))




1.03
